In [1]:
!pip install praw


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 7.4 MB/s eta 0:00:00


In [2]:
import praw

REDDIT_CLIENT_ID = "Ewh28UI9H7w1A4IashZcxg"
REDDIT_SECRET = "FeNXFO-T4DL3Vwd26SPlDE2tD4XhUA"
REDDIT_USER_AGENT = "AskWomenScraper v1.0 by u/Signal_Rich7741"

reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_SECRET,
    user_agent=REDDIT_USER_AGENT
)


In [3]:
#Auth
reddit = praw.Reddit(
    client_id="Ewh28UI9H7w1A4IashZcxg",  # Replace with your actual client ID
    client_secret="FeNXFO-T4DL3Vwd26SPlDE2tD4XhUA",  # Replace with your actual client secret
    user_agent="AskWomenScraper v1.0 by u/Signal_Rich7741"  # Replace with your user agent
)

# Paste my top 20 thread URLs here from the past month:
urls = [
  "https://www.reddit.com/r/AskWomen/comments/1jz543f/women_of_reddit_what_is_the_worst_job_you_ever_had/",
  "https://www.reddit.com/r/AskWomen/comments/1jzlyx7/what_article_of_clothingaccessory_makes_you_feel/",
  "https://www.reddit.com/r/AskWomen/comments/1jz1r17/how_does_being_blonde_or_brunette_affect_the_way/",
  "https://www.reddit.com/r/AskWomen/comments/1jzki9e/what_if_any_safety_precautions_do_you_take_while/",
  "https://www.reddit.com/r/AskWomen/comments/1jy0boc/what_did_emotional_abuse_look_like_in_your/",
  "https://www.reddit.com/r/AskWomen/comments/1jxgi19/when_did_you_experience_your_first_orgasm/",
  "https://www.reddit.com/r/AskWomen/comments/1jvuuhn/what_small_things_tend_to_push_you_over_the_edge/",
  "https://www.reddit.com/r/AskWomen/comments/1jvrvjh/what_do_you_wish_you_knew_before_moving_in_with_a/",
  "https://www.reddit.com/r/AskWomen/comments/1jsyitz/ladies_whats_the_most_ridiculous_lie_a_man_has/",
  "https://www.reddit.com/r/AskWomen/comments/1jqiq6o/women_who_experienced_postpartum_depression_what/",
  "https://www.reddit.com/r/AskWomen/comments/1jq3c50/those_who_have_broken_up_because_you_each_wanted/",
  'https://www.reddit.com/r/AskWomen/comments/1k02gmr/ladies_if_you_were_caught_cheating_by_your/',
  "https://www.reddit.com/r/AskWomen/comments/1jyi6sz/what_helped_get_your_spark_back_after_a_bad_break/",
  "https://www.reddit.com/r/AskWomen/comments/1jy0tx3/married_what_is_your_greatest_regret_about/",
  "https://www.reddit.com/r/AskWomen/comments/1jst9kk/how_often_do_you_look_at_your_partners_phone/",
  "https://www.reddit.com/r/AskWomen/comments/1jpicj1/daughters_whats_something_your_mom_never_told_you/",
  "https://www.reddit.com/r/AskWomen/comments/1jpqaxz/women_who_have_completely_changed_careers_in/",
  "https://www.reddit.com/r/AskWomen/comments/1jpanfc/how_do_you_view_physical_infidelity_vs_emotional/",
  "https://www.reddit.com/r/AskWomen/comments/1jkdgt5/what_is_something_your_mother_raised_you_to/",
  "https://www.reddit.com/r/AskWomen/comments/1jhitks/how_do_you_initiate_intimacy_with_your_partner/"


]

rows = []

for url in urls:
    submission = reddit.submission(url=url)
    submission.comments.replace_more(limit=0)  # Expand all comments

    for comment in submission.comments.list():
        if not comment.author:
            continue
        rows.append({
            "thread_title": submission.title,
            "comment_body": comment.body,
            "comment_author": comment.author.name,
            "created_utc": comment.created_utc,
            "comment_depth": comment.depth,
            "comment_score": comment.score
        })

# Save to CSV
import pandas as pd
df = pd.DataFrame(rows)
df.to_csv("askwomen_manual_top20.csv", index=False)
print("✅ Saved CSV with", len(df), "comments")

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



KeyboardInterrupt: 

In [ ]:
#Install packages for preprocessing

import pandas as pd

!pip install langdetect

!pip install contractions

import contractions


import re
from langdetect import detect

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords # Import stopwords
stop_words = set(stopwords.words('english')) # Set of stopwords in English

# Load your dataset
df = pd.read_csv("askwomen_manual_top20.csv")

# Convert timestamp
df["comment_datetime"] = pd.to_datetime(df["created_utc"], unit='s')

# Save the new version with the datetime column included
df.to_csv("askwomen_preprocessed_with_datetime.csv", index=False)

df = pd.read_csv("askwomen_preprocessed_with_datetime.csv")

df

In [ ]:
#Apply Preprocessing

# 🧼 Step 1: Remove moderator and deleted/removed comments
df = df[
    (~df["comment_author"].str.lower().str.contains("moderator", na=False)) &
    (~df["comment_body"].str.lower().isin(["[deleted]", "[removed]"]))
].copy()


# Step 2: Preprocessing function (preserves emojis, removes Reddit mentions + noise)
def preprocess(text):
    text = text.lower()  # lowercase
    text = contractions.fix(text)  # expand contractions like "i'm" → "i am"
    text = text.strip()  # trim whitespace
    text = re.sub(r"http\S+|www\S+", "", text)  # remove links
    text = re.sub(r"u/\w+", "", text)  # remove u/username
    text = re.sub(r"r/\w+", "", text)  # remove r/subreddit
    text = re.sub(r"[!]{2,}", "!", text)  # collapse !!!
    text = re.sub(r"[?]{2,}", "?", text)  # collapse ???
    text = re.sub(r"[.]{2,}", ".", text)  # collapse ...
    text = ' '.join(word for word in text.split() if word not in stop_words) #remove stopwords here

    return text

df["cleaned_comment"] = df["comment_body"].apply(preprocess)
df


In [ ]:
# 1️⃣ Install and load spaCy
import numpy as np
!pip install spacy
import spacy
from collections import Counter

# Load SpaCy English model
nlp = spacy.load("en_core_web_sm")

# ✅ PART 1: Create noun phrase counter for visual insights
np_counter = Counter()

for comment in df["cleaned_comment"]:
    doc = nlp(comment)
    for np in doc.noun_chunks:
        np_text = np.text.strip().lower()
        if len(np_text.split()) >= 2:  # Only meaningful multi-word phrases
            np_counter[np_text] += 1

# OPTIONAL: View top 20 noun phrases
print(np_counter.most_common(20))


# ✅ PART 2: Create spaCy-filtered + lemmatized comment column
def spacy_filter(text):
    doc = nlp(text)
    tokens = [
        token.lemma_ for token in doc
        if token.pos_ in {"NOUN", "VERB", "ADJ", "ADV"}
        and not token.is_stop
        and not token.is_punct
    ]
    return " ".join(tokens)

# Save result into a new column
df["spacy_filtered"] = df["cleaned_comment"].apply(spacy_filter)

# Preview to confirm
df[["cleaned_comment", "spacy_filtered"]].head()


In [ ]:
# Step 4: Filter out very short comments (<10 chars)
df = df[df["spacy_filtered"].str.len() >= 10].copy()

# Step 5: Detect language, keep all languages for analysis
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

df["lang"] = df["spacy_filtered"].apply(detect_language)

df.to_csv("askwomen_FINAL_preprocessed.csv", index=False)
df = pd.read_csv("askwomen_FINAL_preprocessed.csv")
df


Exploratory Analysis

In [ ]:
# 🟩 Load preprocessed CSV
import pandas as pd
df = pd.read_csv("askwomen_FINAL_preprocessed.csv")

# 📏 Add a column for comment length (based on spacy_filtered now!)
df["len"] = df["spacy_filtered"].str.len()

# 📊 DESCRIPTIVE STATS
print(df["len"].describe())         # Summary stats
print(df["len"].mode().iloc[0])     # Most common length

# 🧑‍💬 AVERAGE COMMENT LENGTH BY AUTHOR (optional)
df.groupby("comment_author")["len"].mean().sort_values(ascending=False).head(10)

# 🕒 TIME SERIES — COMMENTS PER DAY
df["comment_datetime"] = pd.to_datetime(df["comment_datetime"])
daily = df.groupby(df["comment_datetime"].dt.date)["spacy_filtered"].count()

import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
daily.plot(kind="line", marker="o")
plt.title("📅 Comment Frequency Over Time")
plt.xlabel("Date")
plt.ylabel("Number of Comments")
plt.grid(True)
plt.tight_layout()
plt.show()

# 🔠 TOP 20 WORDS (from spacy_filtered)
from collections import Counter
word_counter = Counter()
for comment in df["spacy_filtered"]:
    word_counter.update(comment.split())

top_words = word_counter.most_common(20)

# 📊 BARPLOT
import seaborn as sns
import matplotlib.pyplot as plt

top_words_df = pd.DataFrame(top_words, columns=["word", "count"])
plt.figure(figsize=(10,6))
sns.barplot(data=top_words_df, x="count", y="word", palette="flare")
plt.title("💬 Top 20 Most Frequent Words (SpaCy Filtered)")
plt.xlabel("Frequency")
plt.ylabel("Word")
plt.tight_layout()
plt.show()

# ☁️ WORD CLOUD (from spacy_filtered)
from wordcloud import WordCloud
text = " ".join(df["spacy_filtered"].tolist())
wordcloud = WordCloud(width=1000, height=500, background_color='white', colormap='magma', max_words=200).generate(text)

plt.figure(figsize=(14, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("☁️ Word Cloud of Reddit Comments", fontsize=14)
plt.tight_layout()
plt.show()

# 🌍 LANGUAGE DISTRIBUTION — this part is still okay using cleaned_comment
from langdetect import detect
def detect_lang(text):
    try:
        return detect(text)
    except:
        return "unknown"
df["lang"] = df["spacy_filtered"].apply(detect_lang)

lang_counts = df["lang"].value_counts().head(10)
plt.figure(figsize=(8,5))
lang_counts.plot(kind="bar", color="cornflowerblue")
plt.title("🗣️ Top Languages Used")
plt.xlabel("Language")
plt.ylabel("Number of Comments")
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# 📦 NOUN PHRASES (you already created np_counter earlier using spaCy)
top_nps = pd.DataFrame(np_counter.most_common(20), columns=['Noun Phrase', 'Frequency'])

plt.figure(figsize=(12, 7))
top_nps.sort_values(by='Frequency').plot.barh(x='Noun Phrase', y='Frequency', color='mediumslateblue', legend=False)
plt.title("🔍 Top 20 Most Common Noun Phrases in AskWomen Comments", fontsize=14)
plt.xlabel("Frequency", fontsize=12)
plt.ylabel("Noun Phrase", fontsize=12)
plt.tight_layout()
plt.show()



In [ ]:
#Identify Top Threads on Peak Days of Engagement

# ✅ Step 1: Convert full datetime to date only
df["date_only"] = pd.to_datetime(df["comment_datetime"]).dt.date

# ✅ Step 2: Get top 5 most active dates (highest comment volume)
top_dates = df["date_only"].value_counts().sort_values(ascending=False).head(5)
print("📅 Top 5 Most Active Dates:\n", top_dates)

# ✅ Step 3: Loop through top dates to show most active threads per day
for date in top_dates.index:
    print(f"\n📌 Threads on {date} (Total comments: {top_dates[date]})")
    top_threads = df[df["date_only"] == date]["thread_title"].value_counts().head(5)
    print(top_threads)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Extract just the date from the comment datetime
df["date_only"] = pd.to_datetime(df["comment_datetime"]).dt.date

# Step 2: Identify the top 5 most active dates
top_dates = df["date_only"].value_counts().sort_values(ascending=False).head(5).index

# Step 3: Filter dataset to only include those dates
filtered_df = df[df["date_only"].isin(top_dates)]

# Step 4: Group by date and thread title, count comments
thread_counts = (
    filtered_df.groupby(["date_only", "thread_title"])
    .size()
    .reset_index(name="comment_count")
)

# Optional: Keep only top 5 threads per date for cleaner plot
top_threads_per_day = (
    thread_counts.groupby("date_only")
    .apply(lambda x: x.nlargest(5, "comment_count"))
    .reset_index(drop=True)
)

# Step 5: Plot
plt.figure(figsize=(12, 6))
sns.barplot(
    data=top_threads_per_day,
    x="date_only",
    y="comment_count",
    hue="thread_title",
    dodge=True,
    palette="tab10"
)

plt.title("Top Threads on Most Active Days")
plt.xlabel("Date")
plt.ylabel("Number of Comments")
plt.xticks(rotation=45)
plt.legend(title="Thread Title", bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import matplotlib.pyplot as plt

# Join all spaCy-filtered comments
text = df["spacy_filtered"].dropna().astype(str)

# Vectorize using CountVectorizer
vectorizer = CountVectorizer(max_features=20)
X = vectorizer.fit_transform(text)
df_bow = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Compute correlation matrix
correlation_matrix = df_bow.corr()

# Plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("🔗 Correlation Between Top 20 Words in r/AskWomen Comments")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Extract top N frequent words from spaCy-filtered comments
text = df["spacy_filtered"].dropna().astype(str)
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(text)
words = vectorizer.get_feature_names_out()

# 2. Get sentence embeddings of each word
model = SentenceTransformer("all-MiniLM-L6-v2")
word_embeddings = model.encode(words)

# 3. Reduce dimensions using t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=5)
reduced = tsne.fit_transform(word_embeddings)

# 4. Optional: Cluster words using KMeans
kmeans = KMeans(n_clusters=5, random_state=42)
labels = kmeans.fit_predict(word_embeddings)

# 5. Plot
plt.figure(figsize=(12, 8))
palette = sns.color_palette("Set2", n_colors=5)
for i, word in enumerate(words):
    plt.scatter(reduced[i, 0], reduced[i, 1], color=palette[labels[i]], s=100)
    plt.text(reduced[i, 0]+0.5, reduced[i, 1], word, fontsize=9)

plt.title("🧠 Word Clusters via Semantic Embeddings + t-SNE")
plt.xticks([])
plt.yticks([])
plt.tight_layout()
plt.show()



In [ ]:
#Bar Chart: Most Active Threads (by Comment Volume)

thread_counts = df["thread_title"].value_counts().head(10)

plt.figure(figsize=(12,6))
sns.barplot(y=thread_counts.index, x=thread_counts.values, palette="viridis")
plt.title("🧵 Top 10 Most Commented Threads in AskWomen March-April")
plt.xlabel("Number of Comments")
plt.ylabel("Thread Title")
plt.tight_layout()
plt.show()


In [ ]:
#Which comments resonated most with readers?

#Most upvoted comments top10
top_upvoted = df.sort_values(by="comment_score", ascending=False)[["cleaned_comment", "comment_score"]].head(10)
display(top_upvoted)

# Total number of comments
total_comments = len(df)

# Comments with > 1000 upvotes
viral_comments = df[df["comment_score"] > 1000]
viral_count = len(viral_comments)

# Percentage calculation
viral_percentage = (viral_count / total_comments) * 100
print(f"{viral_count} out of {total_comments} comments ({viral_percentage:.2f}%) received over 1,000 upvotes.")



In [ ]:
# 📊 Average upvote score per thread
df.groupby("thread_title")["comment_score"].mean().sort_values(ascending=False).head(10).plot(
    kind='barh', color='orchid', figsize=(10,6), title="💡 Avg Upvotes per Thread"
)
plt.xlabel("Average Upvote Score")
plt.tight_layout()
plt.show()



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Group by thread and calculate:
# - total comments per thread
# - average upvote score per thread (or total, if you prefer)
thread_stats = df.groupby("thread_title").agg(
    num_comments=("cleaned_comment", "count"),
    avg_upvotes=("comment_score", "mean")
).reset_index()

# Calculate Pearson correlation
correlation = thread_stats["num_comments"].corr(thread_stats["avg_upvotes"])
print(f"Correlation between number of comments and average upvotes: {correlation:.2f}")

# Visualize it
plt.figure(figsize=(10, 6))
sns.scatterplot(data=thread_stats, x="num_comments", y="avg_upvotes", color="orchid")
plt.title("📈 Correlation Between Number of Comments and Average Upvotes per Thread")
plt.xlabel("Number of Comments per Thread")
plt.ylabel("Average Upvotes per Thread")
plt.tight_layout()
plt.show()


#Also calcuate Pearson correlation coefficient
correlation = thread_stats["num_comments"].corr(thread_stats["avg_upvotes"])
print(f"Correlation: {correlation:.2f}")


Sentiment Analysis

In [ ]:
# Create our own sentiment word mapping from VADER
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from collections import defaultdict
import nltk
nltk.download("vader_lexicon")

# Load base VADER lexicon
analyzer = SentimentIntensityAnalyzer()
mapping = dict(analyzer.lexicon)  # Convert from defaultdict to dict for easy editing

# Preview extreme values
{word: score for word, score in mapping.items() if abs(score) > 3}

In [ ]:
import nltk
nltk.download('punkt_tab')
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

# Download necessary tokenizer
#nltk.download('punkt')  # Only this is needed # This is not needed because we're using punkt_tab

# Define the bigram-only sentiment scoring function
def custom_sentiment_score_with_bigrams(text):
    text = text.lower()
    tokens = word_tokenize(text)

    # Generate bigrams only
    bigrams = [' '.join(bigram) for bigram in ngrams(tokens, 2)]

    score = 0
    count = 0
    for gram in bigrams:
        if gram in mapping:  # Uses your adjusted VADER lexicon
            score += mapping[gram]
            count += 1

    return round(score / count, 3) if count > 0 else 0

# Apply to cleaned comment column
df["custom_sentiment_bigram"] = df["cleaned_comment"].apply(custom_sentiment_score_with_bigrams)
df

In [ ]:
# Custom sentiment scoring with bigrams
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
import nltk

# Download the 'punkt_tab' data package
nltk.download('punkt_tab')

def custom_sentiment_score_with_bigrams(text):
    text = text.lower()
    tokens = word_tokenize(text)

    # Create both unigrams and bigrams
    all_grams = tokens + [' '.join(bigram) for bigram in ngrams(tokens, 2)]

    score = 0
    count = 0
    for gram in all_grams:
        if gram in mapping:
            score += mapping[gram]
            count += 1

    return round(score / count, 3) if count > 0 else 0

# Apply to cleaned comments
df["custom_sentiment_bigram"] = df["cleaned_comment"].apply(custom_sentiment_score_with_bigrams)
df

In [ ]:
# Custom scoring based on our mapping
def custom_sentiment_score(text):
    text = text.lower()
    score = 0
    n = 0
    for word in text.split():
        if word in mapping:
            score += mapping[word]
            n += 1
    return round(score / n, 3) if n > 0 else 0

df["custom_sentiment"] = df["cleaned_comment"].apply(custom_sentiment_score)
df


In [ ]:
#Compare averages for custom sentiment and custom sentiment with bigrams

print("Mean Unigram Sentiment:", round(df["custom_sentiment"].mean(), 3))
print("Mean Bigram-Inclusive Sentiment:", round(df["custom_sentiment_bigram"].mean(), 3))

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Unigram sentiment
plt.hist(df["custom_sentiment"], bins=25, alpha=0.6, label="Unigram Sentiment", color='mediumseagreen', edgecolor='black')

# Bigram-inclusive sentiment
plt.hist(df["custom_sentiment_bigram"], bins=25, alpha=0.6, label="Bigram-Inclusive Sentiment", color='mediumpurple', edgecolor='black')

plt.title("📈 Distribution of Custom Sentiment Scores: Unigram vs. Bigram")
plt.xlabel("Sentiment Score")
plt.ylabel("Number of Comments")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()



In [ ]:
import pandas as pd

# Top 5 most positive comments
print("✨ Most Positive Comments:")
print(df.sort_values("custom_sentiment", ascending=False)[["comment_body", "custom_sentiment"]].head(5))

# Top 5 most negative comments
print("\n💔 Most Negative Comments:")
print(df.sort_values("custom_sentiment")[["comment_body", "custom_sentiment"]].head(5))


In [ ]:
import matplotlib.pyplot as plt

# Plot histogram of custom sentiment
df["custom_sentiment"].plot(
    kind='hist',
    bins=20,
    color='mediumseagreen',
    edgecolor='black',
    title="📈 Frequency Distribution of Custom Sentiment Scores"
)
plt.xlabel("Custom Sentiment Score")
plt.ylabel("Number of Comments")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Show mean score
print("🧮 Mean Custom Sentiment Score:", round(df["custom_sentiment"].mean(), 3))


Also score sentiment with TextBlob

In [ ]:
# Install and import TextBlob
from textblob import TextBlob

# Define function to calculate TextBlob polarity score
def get_textblob_sentiment(text):
    return TextBlob(text).sentiment.polarity  # Range: -1 (negative) to +1 (positive)

# Apply to 'cleaned_comment' column
df['textblob_score'] = df['cleaned_comment'].apply(get_textblob_sentiment)

# Plot sentiment score distribution
import matplotlib.pyplot as plt

df["textblob_score"].plot(
    kind='hist',
    bins=20,
    color='mediumseagreen',
    edgecolor='black',
    title="📈 Frequency Distribution of TextBlob Sentiment Scores"
)
plt.xlabel("TextBlob Polarity Score")
plt.ylabel("Number of Comments")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Print mean sentiment score
print("🧮 Mean TextBlob Sentiment Score:", round(df["textblob_score"].mean(), 3))


Now lets compare Custom Sentiment Score and TextBlob

In [ ]:
comparison_df = df[['textblob_score', 'custom_sentiment']]
print(comparison_df.describe())

comparison_df.plot(kind='box', title="📦 Sentiment Score Distribution: TextBlob vs Custom Model")
plt.ylabel("Sentiment Score")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()



In [ ]:
import matplotlib.pyplot as plt

# Plot overlaid histograms
plt.figure(figsize=(10, 6))

# TextBlob histogram
df["textblob_score"].plot(
    kind='hist',
    bins=25,
    alpha=0.6,
    color='mediumseagreen',
    edgecolor='black',
    label='TextBlob Sentiment'
)

# Custom sentiment histogram
df["custom_sentiment"].plot(
    kind='hist',
    bins=25,
    alpha=0.6,
    color='mediumpurple',
    edgecolor='black',
    label='Custom Sentiment'
)

# Plot formatting
plt.title("📊 Overlay: TextBlob vs Custom Sentiment Score Distribution")
plt.xlabel("Sentiment Score")
plt.ylabel("Number of Comments")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Print mean sentiment scores
print("🧮 Mean TextBlob Sentiment Score:", round(df["textblob_score"].mean(), 3))
print("🧮 Mean Custom Sentiment Score:", round(df["custom_sentiment"].mean(), 3))


Topic Modelling LDA

In [ ]:
!pip install gensim

In [ ]:
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd # Import pandas

# Load your dataset - Assuming your CSV is named 'askwomen_FINAL_preprocessed.csv'
df = pd.read_csv("askwomen_FINAL_preprocessed.csv")

# Tokenized spaCy column for LDA
texts = df["spacy_filtered"].dropna().astype(str).str.split()

# Filter out stopwords again (if needed)
stop_words = set(stopwords.words("english"))
texts = [[word for word in doc if word not in stop_words] for doc in texts]

# Create dictionary and corpus
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
#Train LDA model
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=6,       # Change if needed
                     random_state=42,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)


In [ ]:
#Print top words per topic

for i, topic in lda_model.show_topics(num_topics=6, num_words=10, formatted=False):
    print(f"\n🔹 Topic {i+1}:")
    print(", ".join([word for word, prob in topic]))


In [ ]:
#Create a dictionary to label topics
topic_labels = [
    "Reflection & Identity",       # Topic 1
    "Appearance & Gender",         # Topic 2
    "Moderation & Platform",       # Topic 3
    "Relationships & Regret",      # Topic 4
    "Trust & Technology",          # Topic 5
    "Work & Life"                  # Topic 6
]
#Visualize Topic Distribution

import matplotlib.pyplot as plt
import numpy as np

# Get dominant topic per document
topic_distribution = []
for bow in corpus:
    topic_probs = lda_model.get_document_topics(bow, minimum_probability=0.0)
    dominant_topic = max(topic_probs, key=lambda x: x[1])[0]
    topic_distribution.append(dominant_topic)

# Plot histogram
plt.figure(figsize=(8,5))
plt.hist(topic_distribution, bins=range(7), align='left', color="mediumorchid", edgecolor="black")
plt.title("🧠 Topic Distribution Across Reddit Comments")
plt.xlabel("Topic Number")
plt.ylabel("Number of Comments")
plt.xticks(ticks=np.arange(6), labels=topic_labels, rotation=45, ha='right')
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()
